# Cellpose in Python

<div class="custom-button-row">
    <a 
        class="custom-button custom-download-button" href="../../../notebooks/05_segmentation/deep_learning/cellpose_notebook.ipynb" download>
        <i class="fas fa-download"></i> Download this Notebook
    </a>
    <a
    class="custom-button custom-download-button" href="https://colab.research.google.com/github/HMS-IAC/bobiac/blob/gh-pages/colab_notebooks/05_segmentation/deep_learning/cellpose_notebook.ipynb" target="_blank">
        <img class="button-icon" src="../../../_static/logo/icon-google-colab.svg" alt="Open in Colab">
        Open in Colab
    </a>
</div>

In [ ]:
# /// script
# requires-python = ">=3.12"
# dependencies = [
#     "matplotlib",
#     "tifffile",
#     "cellpose"
# ]
# ///

# Overview

In this section...

If you do not have an Apple Silicon Mac or a GPU we suggest to run this notebook in Google Colab. 

...

The images we will use for this section can be downloaded from the <a href="../../../_static/data/05_segmentation_cellpose.zip" download> <i class="fas fa-download"></i> Cellpose Dataset</a>.

## Import Libraries

In [39]:
import tifffile

## Run Cellpose on A Sample Image

...

### Load the Image

...

In [41]:
image_path = "../../../_static/images/cellpose/cell_cellpose.tif"
image = tifffile.imread(image_path)

### Init the Model

...

### Run Cellpose

...

### Display the Results

## Run Cellpose on a Folder of Images

...

TO REMOVE: <a href="https://colab.research.google.com/github/MouseLand/cellpose/blob/main/notebooks/test_Cellpose-SAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>